In [2]:
import torch

t = torch.tensor(range(4), dtype=torch.uint8) + 10
z = torch.tensor([[0, 1, 0, 0],
                  [1, 0, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=torch.uint8)

t @ z

tensor([11, 10, 12, 13], dtype=torch.uint8)

In [3]:
import json
import torch
import numpy as np

with open('data/g.json', 'rb') as file:
    data = json.loads(file.read())

program_data = torch.tensor(data['Program'], dtype=torch.uint8)
character_data = torch.tensor(data['Character'], dtype=torch.uint8)

program_data = np.array(data['Program'])

In [4]:
def SEI(state, a, b): state['StatusRegister']['Interrupt'] = 1
def CLD(state, a, b): state['StatusRegister']['Decimal'] = 0

def Z_set(value):
    state['StatusRegister']['Zero'] = 1*(value == 0)

def N_set(value):
    state['StatusRegister']['Negative'] = 1*(value & 0x80 == 0x80)

def LDA(state, a, b): state['Accumulator'] = a; Z_set(a); N_set(a)
def LDX(state, a, b): state['X'] = a;           Z_set(a); N_set(a)
def LDY(state, a, b): state['Y'] = a;           Z_set(a); N_set(a)
def DEX(state, a, b): state['X'] -= 1;          Z_set(state['X']); N_set(state['X'])

def CMP(state, a, b):
    result = state['Accumulator'] - a
    Z_set(result); N_set(result)
    if a <= state['Accumulator']:
        state['StatusRegister']['Carry'] = 1
    else:
        state['StatusRegister']['Carry'] = 0

def STA(state, a, b): state['Memory'][a] = state['Accumulator']
def TXS(state, a, b): state['StackRegister'] = state['X']

def BPL(state, a, b):
    if state['StatusRegister']['Negative'] == 0:
        state['ProgramCounter'] += np.int8(a)

def BCS(state, a, b):
    if state['StatusRegister']['Carry'] == 1:
        state['ProgramCounter'] += np.int8(a)

def BNE(state, a, b):
    if state['StatusRegister']['Zero'] == 0:
        state['ProgramCounter'] += np.int8(a)

def JSR(state, a, b):
    pass

def imm(state, data):
    return data

def abs_read(state, data1, data2):
    print(f'{data2*256 + data1:04x} -> {state["Memory"][data2*0x0100 + data1]}')

    return state['Memory'][data2*0x0100 + data1]

def abs_x_read(state, data1, data2):
    x = state['X']
    print(f'{data2*256 + data1:04x} + {x:02x} -> {state["Memory"][data2*0x0100 + data1 + x]}')

    return state['Memory'][data2*0x0100 + data1 + x]


In [5]:

_ = lambda x: x

instructions = {
    0x00: ["BRK", ["imp"]],
    0x01: ["ORA", ["X", "ind"]],
    0x10: [BPL, imm],
    0x20: [JSR, abs_read, _],
    0x78: [SEI],
    0x8D: [STA, abs_read, _],
    0x9A: [TXS],
    0xA0: [LDY, imm],
    0xA2: [LDX, imm],
    0xA9: [LDA, imm],
    0xAD: [LDA, abs_read, _],
    0xB0: [BCS, imm],
    0xBD: [LDA, abs_x_read, _],
    0xC9: [CMP, imm],
    0xCA: [DEX],
    0xD0: [BNE, imm],
    0xD8: [CLD]
}

In [6]:
state = {
    'ProgramCounter': 0,
    'Accumulator': 0, 'X': 0, 'Y': 0,
    'StackRegister': 0,
    'StatusRegister': {
        'Negative': 0,
        'Zero': 0,
        'Decimal': 0,
        'Interrupt': 0,
        'Carry': 0
    },
    'ProgramData': program_data,
    # 'Memory': np.zeros(0x0800, dtype=np.uint8)
    'Memory': np.zeros(0x2008, dtype=np.uint8)
}

ops = []

def inc():
    opcode = state['ProgramData'][state['ProgramCounter']]
    state['ProgramCounter'] += 1

    return opcode

def pc(): return state['ProgramCounter']

def vertical_blank(): state['Memory'][0x2002] = 0x80

vertical_blank()

for _1 in range(46):
    index = pc()

    opcode = inc()
    
    operation = instructions[opcode.item()]

    if len(operation) == 1:
        op, = operation
        op(state, 0, 0)

        ops.append([index, f'{opcode:02X} {op.__name__}'])

    elif len(operation) == 2:
        op, f1 = operation
        data = inc()
        
        op(state, f1(state, data), 0)

        ops.append([index, f'{opcode:02X} {op.__name__} .${data:02x}'])

    elif len(operation) == 3:
        op, f1, f2 = operation
        data1, data2 = inc(), inc()
        
        if f2 == _:
            ops.append([index, f'{opcode:02X} {op.__name__} ${data2:02x}{data1:02x} ;{data1*0x0100 + data2:04x}'])
            op(state, f1(state, data1, data2), 0)

opcode = state['ProgramData'][state['ProgramCounter']]

state, opcode.item(), hex(opcode.item())

2000 -> 0
2002 -> 128
2002 -> 128
07d7 + 05 -> 0
07d7 + 04 -> 0
07d7 + 03 -> 0
07d7 + 02 -> 0
07d7 + 01 -> 0
07d7 + 00 -> 0
07ff -> 0


IndexError: index 37068 is out of bounds for axis 0 with size 8200

In [263]:
print('\n' + '\n'.join([f'{index:02X} {op}' for index, op in ops]) + '\n')


00 78 SEI
01 D8 CLD
02 A9 LDA .$10
04 8D STA $2000 ;0020
07 A2 LDX .$ff
09 9A TXS
0A AD LDA $2002 ;0220
0D 10 BPL .$fb
0F AD LDA $2002 ;0220
12 10 BPL .$fb
14 A0 LDY .$fe
16 A2 LDX .$05
18 BD LDA $07d7 ;d707
1B C9 CMP .$0a
1D B0 BCS .$0c
1F CA DEX
20 10 BPL .$f6
18 BD LDA $07d7 ;d707
1B C9 CMP .$0a
1D B0 BCS .$0c
1F CA DEX
20 10 BPL .$f6
18 BD LDA $07d7 ;d707
1B C9 CMP .$0a
1D B0 BCS .$0c
1F CA DEX
20 10 BPL .$f6
18 BD LDA $07d7 ;d707
1B C9 CMP .$0a
1D B0 BCS .$0c
1F CA DEX
20 10 BPL .$f6
18 BD LDA $07d7 ;d707
1B C9 CMP .$0a
1D B0 BCS .$0c
1F CA DEX
20 10 BPL .$f6
18 BD LDA $07d7 ;d707
1B C9 CMP .$0a
1D B0 BCS .$0c
1F CA DEX
20 10 BPL .$f6
22 AD LDA $07ff ;ff07
25 C9 CMP .$a5
27 D0 BNE .$02
2B 20 JSR $90cc ;cc90



In [7]:
np.int8(0xF6)

-10